In [5]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import time
from vardata import years, team_abbr, columns, teams_idx
from results_processor import *
from vardata import teams

year = '2023'
week = '10'
url = 'https://www.pro-football-reference.com'
stats = pd.read_csv('stats/' + year + '/' + 'stats_'+ year + '_week_' + week + '.csv', index_col=[0])
games = pd.read_csv('stats/' + year + '/' + 'games_'+ year + '_week_' + week + '.csv', index_col=[0])


In [6]:
final_data = pd.read_csv('stats/final_data.csv', index_col=[0])

In [7]:
def scrape_games(year, week):
    
    # use beautiful soup to return the html of the webpage
    r = requests.get(url + '/years/'+ year + '/games.htm')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # grab the game results table from the webpage
    game_stats = soup.find_all('table')[0]

    # collect table rows
    stat_rows = game_stats.findAll('tr')
    
    # initalize 
    results = []
    n = 0
    
    # looping through the table rows to collect data
    for i in range(len(stat_rows)):
        if n >= 19:
            wait = random.randint(65, 120)
            time.sleep(wait)
            n = 0
            
        if stat_rows[i].find('th').getText() != week:
            continue

        row = []
        row.extend([col.getText() for col in stat_rows[i].findAll('td')])
        
        # handling home / away team info being in different locations for each row
        if row[4] != '@':
            home_team = teams[row[3]]
            away_team = teams[row[5]]
            ht_score = int(row[7])
            at_score = int(row[8])
            total_points = ht_score + at_score
            ht_pts_diff = ht_score - at_score

        else:
            home_team = teams[row[5]]
            away_team = teams[row[3]]
            ht_score = int(row[8])
            at_score = int(row[7])
            total_points = ht_score + at_score
            ht_pts_diff = ht_score - at_score

        # appending necessary info to results array
        results.append([home_team, away_team, ht_score, at_score, total_points,
                        ht_pts_diff])

        n += 1
            
    return results

In [8]:
results = scrape_games(year, week)

In [9]:
results = pd.DataFrame(results, columns = ['home_team', 'away_team', 'ht_score',
                                          'at_score', 'total_points', 'ht_pts_diff'])

In [10]:
stats['ht_score'] = results['ht_score']
stats['at_score'] = results['at_score']
stats['total_points'] = results['total_points']
stats['ht_pts_diff'] = results['ht_pts_diff']
stats['ht_vegas_line'] = games['ht_vegas_line']

In [11]:
games['ht_pts_diff'] = results['ht_pts_diff']
games = games.rename(columns={"ht_point_differential": "model_pred"})

In [12]:
games.head()

,home_team,away_team,ht_vegas_line,model_pred,ht_pts_diff
0,chi,car,-3.0,6.207019,3
1,nwe,clt,2.0,-1.655258,-4
2,rav,cle,-6.0,8.075488,-3
3,pit,gnb,-4.0,3.300674,-2
4,cin,htx,-7.0,2.513752,4


In [13]:
process_results(games)

Vegas Line:  -4.0 , model pred:  -3 , final pts diff:  -2
Vegas Line:  -7.0 , model pred:  -3 , final pts diff:  4
Vegas Line:  0.0 , model pred:  -1 , final pts diff:  14
Vegas Line:  3.0 , model pred:  -2 , final pts diff:  8
Vegas Line:  3.0 , model pred:  0 , final pts diff:  2
Vegas Line:  -6.0 , model pred:  -4 , final pts diff:  3
Vegas Line:  -1.0 , model pred:  -4 , final pts diff:  4
Total Number of Games:
 14
Winning Bets Count:
 7
No Bet Placed Count:
 2
Winning Percentage:
 0.5833
______________________________________
Winner Correctly Predicted Count:
 9
Accuracy in Classifaction:
 0.6429
______________________________________
Mean Squared Error of Results:
 145.6608
Root Mean Squared Error of Results:
 12.0690


In [14]:
final_data = final_data.append(stats, ignore_index=True)

/var/folders/jw/95zp9jr92y72x0m_9wppzqmw0000gn/T/ipykernel_71462/1502454349.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append(stats, ignore_index=True)


In [15]:
games.to_csv('stats/' + year + '/' + 'games_'+ year + '_week_' + week + '.csv')
stats.to_csv('stats/' + year + '/' + 'stats_' + year + '_week_' + week + '.csv')
final_data.to_csv('stats/final_data.csv')